In [2]:
import pandas as pd
import torch

diabetes_df = pd.read_csv("../diabetes.csv")
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = diabetes_df.drop('Outcome', axis=1).values
y = diabetes_df['Outcome'].values

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

# #Standardize
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [1]:
import torch.nn as nn
import torch.nn.functional as F #this has activation functions

# Creating tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

print(X_train)

NameError: name 'torch' is not defined

In [5]:
class ANN_Model(nn.Module):
    def __init__(self, input_features=8, hidden1=20, hidden2=20, out_features =2):
        super().__init__()
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
    
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [24]:
torch.manual_seed(42)

#instantiate the model
model = ANN_Model()

In [25]:
# loss function
loss_function = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.Rprop(model.parameters(), lr = 0.01)

In [26]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = model.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss.item()}')
    
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() #for backward propagation 
    optimizer.step() #performs one optimization step each epoch
    

Epoch number: 1 with loss: 0.647413969039917
Epoch number: 11 with loss: 0.4519883096218109
Epoch number: 21 with loss: 0.38703450560569763
Epoch number: 31 with loss: 0.33954641222953796
Epoch number: 41 with loss: 0.30658775568008423
Epoch number: 51 with loss: 0.2752005457878113
Epoch number: 61 with loss: 0.25215277075767517
Epoch number: 71 with loss: 0.22651635110378265
Epoch number: 81 with loss: 0.20444047451019287
Epoch number: 91 with loss: 0.1857936829328537
Epoch number: 101 with loss: 0.17048321664333344
Epoch number: 111 with loss: 0.1541174203157425
Epoch number: 121 with loss: 0.13827122747898102
Epoch number: 131 with loss: 0.1239677220582962
Epoch number: 141 with loss: 0.11416147649288177
Epoch number: 151 with loss: 0.10690715163946152
Epoch number: 161 with loss: 0.10015998780727386
Epoch number: 171 with loss: 0.09251971542835236
Epoch number: 181 with loss: 0.08521825820207596
Epoch number: 191 with loss: 0.07992041110992432
Epoch number: 201 with loss: 0.0756181

In [27]:
#predictions
y_pred = []

with torch.no_grad():
    for i, data in enumerate(X_test):
        prediction = model(data)
        y_pred.append(prediction.argmax().item())



In [28]:
from sklearn.metrics import accuracy_score
a_score = accuracy_score(y_test, y_pred)
print(a_score)

0.7142857142857143


In [29]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.77      0.78       100
           1       0.59      0.61      0.60        54

    accuracy                           0.71       154
   macro avg       0.69      0.69      0.69       154
weighted avg       0.72      0.71      0.72       154



## From DataCamp when learning how to use Keras
### It is suggested to scale the data prior to fitting it so each feature is, on average, about similar sized values. One common approach is to subtract each feature by that features mean, and divide it by it's standard deviation.

For Keras classifier function you want to set the loss to categorical_crossentropy, may also want to add metrics=['accuracy'] when you specify loss because it's easier to interpret than the crossentropy. And then of course a classification report
You may also want to use the softmax activation for the final output instead of relu. Relu is still used for hidden layers
Classification as an output node for each outcome instead of just one output node for regression
For keras instead of having one column for the target you will split out to a column for each target outcome

For regressions you use mean_squared_error